# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

In [1]:
# Run this cell first!

from helpers import Map, load_map, show_map
from student_code import shortest_path

%load_ext autoreload
%autoreload 2

### Map Basics

In [3]:
map_10 = load_map('map-10.pickle')
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections

{0: [0.7798606835438107, 0.6922727646627362],
 1: [0.7647837074641568, 0.3252670836724646],
 2: [0.7155217893995438, 0.20026498027300055],
 3: [0.7076566826610747, 0.3278339270610988],
 4: [0.8325506249953353, 0.02310946309985762],
 5: [0.49016747075266875, 0.5464878695400415],
 6: [0.8820353070895344, 0.6791919587749445],
 7: [0.46247219371675075, 0.6258061621642713],
 8: [0.11622158839385677, 0.11236327488812581],
 9: [0.1285377678230034, 0.3285840695698353]}

In [4]:
map_10.intersections[0]

[0.7798606835438107, 0.6922727646627362]

**Roads**

The `roads` property is a list where, if `i` is an intersection, `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [5]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [6]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [4, 3, 1],
 [5, 4, 1, 2],
 [1, 2, 3],
 [7, 0, 3],
 [0],
 [0, 5],
 [9],
 [8]]

In [4]:
# map_40 is a bigger map than map_10
map_40 = load_map('map-40.pickle')
show_map(map_40)

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [8]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=8, goal=24, path=[8, 33, 30, 14, 5, 16, 37, 12, 34, 31, 10, 24])

### Writing your algorithm
You should open the file `student_code.py` in another tab and work on your algorithm there. Do that by selecting `File > Open` and then selecting the appropriate file.

The algorithm you write will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`

```bash
> shortest_path(map_40, 5, 34)
[5, 16, 37, 12, 34]
```

In [34]:
import math
from collections import defaultdict
import heapq

def shortest_path(M, start, goal):
    explored_id = list()
    explored_id.append(start)
    explored_distance = 0

    current_id = start
    previous_id = start
    terminate_id = start
    next_id = start
    itera = 0

    if start == goal:
        print([start])

    established = {}

    test = defaultdict()

    #############
    explored_set = list()
    explored_set.append(start)
    ############
    
    choice_distance_priority_queue = list()
    
    while current_id != goal:




        ################
        if current_id in explored_set:
            explored_set.remove(current_id)
        ###############    

        if len(explored_id) == 1:
            previous_id = start
        else:
            previous_id = explored_id[-2]

        #print("\n\n======itera", itera, "begin,", "explored", explored_id)

        # get neighbour id for current id    
        all_neighbour_id = M.roads[current_id]

        #frontier_id = [x for x in all_neighbour_id if x not in explored_id]


        frontier_id = set()
        for x in all_neighbour_id:
            if x not in explored_id:
                frontier_id.add(x)

        # record of total distance of current id:
        #choice_distance = {}
        
        
        #print("itera", itera, "middle",", frontier_id:", frontier_id)
        #print("previous_id,current_id",previous_id, current_id)
        explored_distance += distance(M, previous_id, current_id)
        #print("explored_distance", explored_distance)



        for node_id in frontier_id:
            # use A* algorithm, distance = greedy distance + heuristic distance
            greedy_distance = []
            greedy_distance.append(node_id)
            greedy_distance += explored_id

            #print("\n  --------exploring frontier node_id ", node_id)
            #print("  greedy_distance_id", greedy_distance, ",\n  explored_id:", greedy_distance[1:])


            #test.append([current_id, node_id])
            #print("test0: ", test)

            #print("explored_set: ", opeexplored_setnset)
            #if node_id not in explored_set or (len(explored_set) == 1 and node_id in explored_set):
            if node_id not in explored_set or len(explored_set) == 1:
                #print("test1: ", test)
                #print("++++++++++++++++++   current_id" , current_id)
                test[node_id] = current_id
                #print("New, current",node_id,current_id ,"added !!!!!!!!!!!!!!")
                explored_set.append(node_id)


            g_distance = explored_distance + distance(M, current_id, node_id)
            #g_distance = distance(M, current_id, node_id)
            h_distance = distance(M, node_id, goal)
            #print ("  pre h_distance: ",h_distance)


            if h_distance > distance(M, current_id, goal):
                h_distance = distance(M, current_id, goal)
            else:
                h_distance = distance(M, node_id, goal)


            #print("  after h_distance: ", h_distance)

            total_distance = 1 * g_distance + h_distance

            #choice_distance[node_id] = total_distance
            heapq.heappush(choice_distance_priority_queue, (total_distance, node_id))
            
            established[node_id] = total_distance
            
            '''
            print("\n  explored_distance:",round(explored_distance,3),\
                  "\n  candidate_greedy_distance between ",current_id, "and", node_id,"is", round(distance(M, current_id, node_id),3),\
                  "\n  g_distance: ", round(g_distance,3),\
                 "\n  final h_distance between ",node_id, "and", goal,"is", round(h_distance,3),\
                 "\n  total_distance:", round(total_distance,3))
                      
            '''
            

        # return min value (total distance) of choice_distance:
        #next_id = min(choice_distance, key=choice_distance.get)
        if len(choice_distance_priority_queue) == 0:
            print("Path not found ! Return retrived path: ")
            break
        else:
            (_, next_id) = heapq.heappop(choice_distance_priority_queue)
        
        #print("\n  choice_distance dictionary:\n", choice_distance)
        #print("  established: \n", established)

        #print("gallery: ", gallery)

        #print("\n  current_id:", current_id, ", frontier_id:", frontier_id)
        #print("\n----------exploring completed, choose id: ", next_id)
        explored_id.append(next_id)

        #print("\nitera", itera, "end, explored_id:", explored_id,"\n============")
        current_id = next_id
        itera += 1

    #print("goal achieved!")
    #print(explored_id)
    #print("test: ",test)
    return path_retrieve(test, current_id)

# distance between two points on the map M   
def distance(M, a, b):
    '''
    Input: two id a, b in integer
    Output: distance between a and b
    '''
    # distance = sqrt((delta_x)^2 + (delta_y)^2)
    return math.sqrt((M.intersections[a][0] - M.intersections[b][0])**2 + (M.intersections[a][1] - M.intersections[b][1])**2)

def path_retrieve(test, current_id):
    path = [current_id]
    while current_id in test.keys():
        current_id = test[current_id]
        path.append(current_id)
    return path[::-1]

In [35]:
shortest_path(map_40, 8, 24)

[8, 14, 16, 37, 12, 17, 10, 24]

In [36]:
shortest_path(map_10, 3, 9)

Path not found ! Return retrived path: 


[3, 5, 7, 0, 6]

In [37]:
shortest_path(map_40, 5, 34)

[5, 16, 37, 12, 34]

In [11]:
path = shortest_path(map_40, 5, 34)
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

great! Your code works for these inputs!


In [12]:
path = shortest_path(map_40, 8, 24)
if path == [8, 14, 16, 37, 12, 31, 10, 24]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

something is off, your code produced the following:
[8, 14, 16, 37, 12, 17, 10, 24]


In [13]:
path = shortest_path(map_40, 5, 5)
if path == [5]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

[5]
great! Your code works for these inputs!


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [10]:
from test import test

test(shortest_path)

[5]
All tests pass! Congratulations!


In [11]:
shortest_path(map_40, 8, 24)

[8, 14, 16, 37, 12, 31, 10, 24]